In [32]:
import sys
sys.path.append("/home/brunoperdigao/repos/Plan-Classification-Topologic-DGL")

In [33]:
# Import Libraries
from topologicpy.DGL import DGL
from topologicpy.Dictionary import Dictionary
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
import itertools
import dgl
import torch
import random
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
from dgl import save_graphs, load_graphs
from tqdm.notebook import tqdm

In [34]:
import dgl

In [35]:
pwd

'/home/bruno_perdigao/repos/Plan-Classification-Topologic-DGL'

In [36]:
path = "./data2"
dataset = DGL.DatasetByCSVPath(path, numberOfGraphClasses=0,
                           nodeATTRKey='feat', edgeATTRKey='feat',
                           nodeOneHotEncode=False, nodeFeaturesCategories=[],
                           edgeOneHotEncode=False, edgeFeaturesCategories=[], addSelfLoop=False)
print(dataset)

Done saving data into cached files.
Dataset("GraphDGL", num_graphs=200, save_path=/home/bruno_perdigao/.dgl/GraphDGL)


In [37]:
train_data, val_data, test_data = dgl.data.utils.split_dataset(dataset, [0.7, 0.1, 0.2])

In [38]:
train_data.dataset

Dataset("GraphDGL", num_graphs=200, save_path=/home/bruno_perdigao/.dgl/GraphDGL)

---

In [39]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [40]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [14]:
# # print(len(dataset.graphs))
# for i, graph in enumerate(dataset.graphs):
#     # print(i)
#     node_features = graph.ndata['feat']
#     # print(node_features)
#     node_labels = graph.ndata['label']
#     # print(node_labels)
#     train_mask = graph.ndata['train_mask']
#     valid_mask = graph.ndata['val_mask']
#     test_mask = graph.ndata['test_mask']
#     n_features = node_features.shape[1]
#     # print(n_features)
#     n_labels = int(node_labels.max().item() + 1)
#     # print(n_features)

In [43]:
n_features = 5
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=14)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    for i, graph in enumerate(dataset.graphs):
        node_features = graph.ndata['feat']
        node_labels = graph.ndata['label']
        train_mask = graph.ndata['train_mask']
        valid_mask = graph.ndata['val_mask']
        test_mask = graph.ndata['test_mask']
        n_features = node_features.shape[1]
        n_labels = int(node_labels.max().item() + 1)
        
        model.train()
        # forward propagation by using all nodes
        logits = model(graph, node_features)
        # compute loss
        loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
        # compute validation accuracy
        acc = evaluate(model, graph, node_features, node_labels, valid_mask)
        print("acurracy:", acc)
        # backward propagation
        opt.zero_grad()
        loss.backward()
        opt.step()
        print(loss.item())
    
        # Save model if necessary.  Omitted in this example.

acurracy: 0.0
21.628433227539062
acurracy: 0.0
23.049184799194336
acurracy: 0.0
17.703269958496094
acurracy: 0.0
17.388263702392578
acurracy: 0.3333333333333333
15.061763763427734
acurracy: 0.0
11.532702445983887
acurracy: 0.0
10.140252113342285
acurracy: 0.3333333333333333
11.961884498596191
acurracy: 0.3333333333333333
8.552093505859375
acurracy: 0.0
6.402406692504883
acurracy: 0.6666666666666666
2.7777445316314697
acurracy: 0.3333333333333333
4.202413082122803
acurracy: 0.75
4.738986015319824
acurracy: 0.3333333333333333
3.650498628616333
acurracy: 0.6666666666666666
1.4501358270645142
acurracy: 0.5
6.364037036895752
acurracy: 0.6666666666666666
3.4837253093719482
acurracy: 0.6666666666666666
6.8120832443237305
acurracy: 0.3333333333333333
4.023338317871094
acurracy: 0.6666666666666666
1.8755688667297363
acurracy: 0.6666666666666666
3.416745901107788
acurracy: 0.3333333333333333
5.5723066329956055
acurracy: 0.6666666666666666
3.396989583969116
acurracy: 0.6666666666666666
2.79285526

In [44]:
DGL.ModelClassifyNodes(model, test_data.dataset)

{'alllabels': [[13,
   13,
   12,
   12,
   11,
   12,
   13,
   12,
   2,
   12,
   13,
   13,
   0,
   13,
   13,
   3,
   13,
   7,
   4,
   12,
   1,
   12,
   12,
   13,
   8,
   12,
   13,
   6,
   9,
   13],
  [12,
   12,
   6,
   12,
   13,
   13,
   9,
   2,
   4,
   7,
   6,
   13,
   3,
   8,
   0,
   1,
   13,
   13,
   13,
   11,
   13,
   13,
   12,
   13,
   12,
   12,
   13,
   7,
   13,
   13,
   12,
   13,
   12],
  [12,
   13,
   1,
   12,
   11,
   13,
   13,
   13,
   0,
   4,
   12,
   13,
   13,
   12,
   13,
   12,
   6,
   12,
   13,
   7,
   13,
   7,
   12,
   13,
   12,
   8,
   6,
   9,
   13,
   3,
   2,
   13,
   13],
  [13,
   12,
   1,
   13,
   12,
   0,
   12,
   13,
   13,
   13,
   12,
   12,
   13,
   13,
   13,
   13,
   8,
   3,
   13,
   6,
   13,
   4,
   13,
   12,
   2,
   6,
   7,
   12,
   12,
   13,
   7,
   11,
   9],
  [0,
   13,
   12,
   12,
   13,
   12,
   7,
   13,
   13,
   12,
   12,
   13,
   13,
   4,
   13,
   13,
   13,
   12,

In [45]:
dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
# actual = dictionary['alllabels']
actual = []
for l in dictionary['alllabels']:
    actual.extend(l)
    
# print(actual)
# predicted = dictionary['allpredictions']
predicted = []
for l in dictionary['allpredictions']:
    predicted.extend(l)
# print(predicted)
result = DGL.Accuracy(actual=actual, predicted=predicted)
# print(result)
print("Accuracy:", result['accuracy'])

Accuracy: 0.873012


In [48]:
result = DGL.Precision(actual=actual, predicted=predicted)
result

0.873012

In [75]:
cf = DGL.ConfusionMatrix(actual = actual, predicted = predicted)


for i, pred in enumerate(cf):
    true_positive = 0
    false_positive = 0
    false_negative = 0
    true_positive = pred[i]
    false_positive = sum([*pred]) - pred[i]
    for j, pred2 in enumerate(cf):
        false_negative += pred2[i]

    false_negative -= true_positive
    print(i)
    print(true_positive, false_positive, false_negative)

0
150 13 0
1
0 0 130
2
40 0 200
3
0 0 200
4
40 0 160
5
0 0 70
6
392 503 0
7
445 242 0
8
18 68 182
9
92 39 62
10
230 174 0
11
150 39 74
12
3078 0 0
13
2776 0 0


In [58]:
cf = DGL.ConfusionMatrix(actual = actual, predicted = predicted)
print(cf)
fig = Plotly.FigureByConfusionMatrix(cf)
Plotly.Show(fig)

[[ 150    0    0    0    0    0    0    0    0   13    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0   40    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0   40    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0  200  191    0   70  392    0    0    2    0   40    0    0]
 [   0    0    0    0   90    0    0  445  152    0    0    0    0    0]
 [   0    0    0    0   34    0    0    0   18    0    0   34    0    0]
 [   0    3    0    0   36    0    0    0    0   92    0    0    0    0]
 [   0  127    0    0    0    0    0    0    0   47  230    0    0    0]
 [   0    0    0    9    0    0    0    0   30    0    0  150    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0 3078    0]
 [   0    0    0    0    0    0    0    0    0    0

# Grid Search

In [30]:
def evaluate_selected_model(model):
    dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
    # actual = dictionary['alllabels']
    actual = []
    for l in dictionary['alllabels']:
        actual.extend(l)
        
    # print(actual)
    # predicted = dictionary['allpredictions']
    predicted = []
    for l in dictionary['allpredictions']:
        predicted.extend(l)
    # print(predicted)
    acc = DGL.Accuracy(actual=actual, predicted=predicted)
    precision = DGL.Precision(actual=actual, predicted=predicted)
    recall = DGL.Recall(actual=actual, predicted=predicted)
    # print(result)
    return acc, precision, recall

class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, num_layers, activation):
        super().__init__()
        self.layers = nn.ModuleList()
        self.activation = activation
        
        # Create the specified number of hidden layers
        self.layers.append(dglnn.SAGEConv(in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean'))
        for _ in range(num_layers - 1):
            self.layers.append(dglnn.SAGEConv(in_feats=hid_feats, out_feats=hid_feats, aggregator_type='mean'))
        self.layers.append(dglnn.SAGEConv(in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean'))

    def forward(self, graph, inputs):
        h = inputs
        for layer in self.layers[:-1]:
            h = layer(graph, h)
            h = self.activation(h)  # Apply activation function
        h = self.layers[-1](graph, h)  # Last layer without activation
        return h

# Define hyperparameter grid

num_hidden_layers = [2, 5]
num_units_per_layer = [5, 100]
activation_functions = [F.relu, F.tanh]
learning_rates = [0.01,0.0001]
batch_sizes = [32, 64]

# num_hidden_layers = [2, 3, 4, 5]
# num_units_per_layer = [5, 25, 50, 100]
# activation_functions = [F.relu, F.tanh]
# learning_rates = [0.01, 0.0005, 0.001, 0.0001]
# batch_sizes = [16, 32, 64]

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes))

results = []
# Iterate over all combinations of hyperparameters
for num_layers, hid_feats, activation, lr, batch_size in tqdm(param_grid):
    print(f"Training with layers: {num_layers}, units per layer: {hid_feats}, activation: {activation.__name__}, learning rate: {lr}, batch size: {batch_size}")
    
    # Initialize model and optimizer
    n_features = 5
    model = SAGE(in_feats=n_features, hid_feats=hid_feats, out_feats=14, num_layers=num_layers, activation=activation)
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(20):  # You can adjust the number of epochs as needed
        for i, graph in enumerate(dataset.graphs):
            node_features = graph.ndata['feat']
            node_labels = graph.ndata['label']
            train_mask = graph.ndata['train_mask']
            valid_mask = graph.ndata['val_mask']
            test_mask = graph.ndata['test_mask']
            n_features = node_features.shape[1]
            n_labels = int(node_labels.max().item() + 1)

            model.train()
            # Forward propagation
            logits = model(graph, node_features)
            # Compute loss
            loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
            # Compute validation accuracy
            acc = evaluate(model, graph, node_features, node_labels, valid_mask)
            # Backward propagation
            opt.zero_grad()
            loss.backward()
            opt.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}, Validation Accuracy: {acc}")

    # Optionally, you can save the model or results for each combination
    acc, precision, recall = evaluate_selected_model(model)
    result = {
        'num_hidden_layers': num_layers,
        'num_units_per_layer': hid_feats,
        'activation_functions': activation,
        'learning_rates': lr, 
        'batch_sizes': batch_size,
        'acc': acc['accuracy'],
        'precision': precision,
        'recall': recall,
    }
    results.append(result)



  0%|          | 0/32 [00:00<?, ?it/s]

Training with layers: 2, units per layer: 5, activation: relu, learning rate: 0.01, batch size: 32
Epoch 0, Loss: 0.3758317530155182, Validation Accuracy: 0.75
Epoch 1, Loss: 0.2585024833679199, Validation Accuracy: 0.75
Epoch 2, Loss: 0.18223297595977783, Validation Accuracy: 0.75
Epoch 3, Loss: 0.14460663497447968, Validation Accuracy: 0.75
Epoch 4, Loss: 0.12252325564622879, Validation Accuracy: 0.75
Epoch 5, Loss: 0.1137142926454544, Validation Accuracy: 0.75
Epoch 6, Loss: 0.10637130588293076, Validation Accuracy: 0.75
Epoch 7, Loss: 0.11185094714164734, Validation Accuracy: 0.75
Epoch 8, Loss: 0.11741354316473007, Validation Accuracy: 0.75
Epoch 9, Loss: 0.10370291024446487, Validation Accuracy: 0.75
Epoch 10, Loss: 0.09740213304758072, Validation Accuracy: 0.75
Epoch 11, Loss: 0.07783523947000504, Validation Accuracy: 0.75
Epoch 12, Loss: 0.07242736965417862, Validation Accuracy: 0.75
Epoch 13, Loss: 0.08725149929523468, Validation Accuracy: 0.75
Epoch 14, Loss: 0.09477064758539

In [31]:
results

[{'num_hidden_layers': 2,
  'num_units_per_layer': 5,
  'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  'learning_rates': 0.01,
  'batch_sizes': 32,
  'acc': 0.863235,
  'precision': 0.863235,
  'recall': 0.863235},
 {'num_hidden_layers': 2,
  'num_units_per_layer': 5,
  'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  'learning_rates': 0.01,
  'batch_sizes': 64,
  'acc': 0.815997,
  'precision': 0.815997,
  'recall': 0.815997},
 {'num_hidden_layers': 2,
  'num_units_per_layer': 5,
  'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  'learning_rates': 0.0001,
  'batch_sizes': 32,
  'acc': 0.492049,
  'precision': 0.492049,
  'recall': 0.492049},
 {'num_hidden_layers': 2,
  'num_units_per_layer': 5,
  'activation_functions': <function torch.nn.functional.relu(input: tor